# FAA Aircraft Registration Database

In [1]:
import pandas as pd
import requests
import matplotlib as mpl
import geopandas as gpd
import descartes
import geojson
import json
import numpy as np
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_colwidth', 100)

In [2]:
# https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/

In [3]:
master = pd.read_csv('/Users/mhustiles/data/data/FAA_Aircraft_Database/MASTER.txt', low_memory=False)

In [4]:
reference = pd.read_csv('/Users/mhustiles/data/data/FAA_Aircraft_Database/ACFTREF.txt', low_memory=False)

In [5]:
master.columns = master.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [6]:
reference.columns = reference.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [7]:
reference.loc[44307]

code                                     0563500
mfr               GREGORY T HEDDERLY            
model                       PJ260 SENIOR AERO SP
type_acft                                      4
type_eng                                       8
ac_cat                                         1
build_cert_ind                                 1
no_eng                                         1
no_seats                                       2
ac_weight                                CLASS 1
speed                                          0
unnamed:_11                                  NaN
Name: 44307, dtype: object

In [8]:
master.name = master.name.str.strip()
master.certification = master.certification.str.strip()
master.type_registrant = master.type_registrant.str.strip()

In [9]:
owners = master.groupby(['name', 'type_registrant', 'certification']).agg('size').reset_index(name='planes')

In [10]:
owners.head()

,name,type_registrant,certification,planes
0,,,,160
1,,,1,33
2,,,1B,2
3,,,1N,37
4,,,1NA,2


In [11]:
corptrans = owners[(owners['planes'] > 20) & (owners['certification'] == '1T')]

In [12]:
airlines = ['DELTA AIR LINES INC', 'AMERICAN AIRLINES INC', 'SKYWEST AIRLINES INC', 'SWIFT AIR LLC',\
'UNITED AIRLINES INC', 'SOUTHWEST AIRLINES CO', 'JETBLUE AIRWAYS CORP', 'HORIZON AIR INDUSTRIES',\
'ALASKA AIRLINES INC', 'REPUBLIC AIRWAYS INC', 'SPIRIT AIRLINES INC', 'MESA AIRLINES INC']

In [13]:
airlines

['DELTA AIR LINES INC',
 'AMERICAN AIRLINES INC',
 'SKYWEST AIRLINES INC',
 'SWIFT AIR LLC',
 'UNITED AIRLINES INC',
 'SOUTHWEST AIRLINES CO',
 'JETBLUE AIRWAYS CORP',
 'HORIZON AIR INDUSTRIES',
 'ALASKA AIRLINES INC',
 'REPUBLIC AIRWAYS INC',
 'SPIRIT AIRLINES INC',
 'MESA AIRLINES INC']

In [14]:
cargo = ['UNITED PARCEL SERVICE CO','FEDERAL EXPRESS CORP', 'FEDERAL EXPRESS CORPORATION',\
            'CARGO AIRCRAFT MANAGEMENT INC', 'ATLAS AIR INC']

In [15]:
cargo

['UNITED PARCEL SERVICE CO',
 'FEDERAL EXPRESS CORP',
 'FEDERAL EXPRESS CORPORATION',
 'CARGO AIRCRAFT MANAGEMENT INC',
 'ATLAS AIR INC']

In [16]:
private = ['NETJETS SALES INC', 'FLEXJET LLC']

In [17]:
corptrans.sort_values(by='planes', ascending=False).head()

,name,type_registrant,certification,planes
220089,WELLS FARGO TRUST CO NA TRUSTEE,3,1T,1231
54055,DELTA AIR LINES INC,3,1T,979
8943,AMERICAN AIRLINES INC,3,1T,910
211582,UNITED AIRLINES INC,3,1T,861
16036,BANK OF UTAH TRUSTEE,3,1T,787


In [18]:
airline_planes = master

In [19]:
airline_planes_slim = airline_planes[['name','n_number']]

In [20]:
airline_planes_slim.to_csv('output/airline_planes.csv', index=False)

### Airline planes

In [21]:
american = master[master['name'].str.contains('AMERICAN AIRLINES')]

In [22]:
american.iloc[583]

n_number                                                         766AN
serial_number                           32880                         
mfr_mdl_code                                                   1385213
eng_mfr_mdl                                                      54555
year_mfr                                                          2003
type_registrant                                                      3
name                                             AMERICAN AIRLINES INC
street                               1 SKYVIEW DR                     
street2                              MD 8B369                         
city                                                FORT WORTH        
state                                                               TX
zip_code                                                    76155     
region                                                               2
county                                                             439
countr

In [23]:
master['n_number'] = 'N' + master['n_number']

In [24]:
src = master.merge(reference, left_on='mfr_mdl_code', right_on='code')

In [25]:
src[src['n_number'] == 'N302FD']

,n_number,serial_number,mfr_mdl_code,eng_mfr_mdl,year_mfr,type_registrant,name,street,street2,city,...,model,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,unnamed:_11
207667,N302FD,41239,0260106,52174,2011,5,LOS ANGELES CITY,C/O LOS ANGELES FIRE DEPARTMENT,AIR OPS SECTION 16617 ARMINTA ST,VAN NUYS,...,AW139,6,3,1,0,2,17,CLASS 3,0,nan


In [26]:
lapd = ['AIR SUPPORT DIVISION', 
        'LAPD', 
        'LOS ANGELES POLICE', 'LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION']

In [27]:
lapd_county = ['LOS ANGELES COUNTY',
               'LOS ANGELES CITY',
        'LA COUNTY',
        'CONSOLIDATED FIRE PROTECTION', 
        'LACOUNTY', 
        'LAPD', 
        'LOS ANGELES POLICE',
               'LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION',
       'COUNTY OF LOS ANGELES',
       'COUNTY OF LA',
              'CITY OF LOS ANGELES',
              'LOS ANGELES FIRE',
              'LOS ANGELES FIRE DEPARTMENT',
              'LOS ANGELES CITY']

In [28]:
la_cops_choppers = src[(src['name'].str.contains('|'.join(lapd_county))) \
                       & (src['state'] == 'CA') \
              & (src['type_aircraft'] == '6')]

In [29]:
la_safety_choppers_slim = la_cops_choppers\
    [[ 'n_number', 'name', 'mfr','model','street','city','year_mfr','no_seats' ]]

In [30]:
len(la_cops_choppers)

55

In [31]:
la_cops_choppers.to_csv('output/la_safety_choppers_slim.csv', index=False)

In [32]:
la_cops_choppers.n_number.unique().tolist()

['N306FD',
 'N664PD',
 'N3202Q',
 'N601CC',
 'N120LA',
 'N73985',
 'N213PF',
 'N221LA',
 'N223LA',
 'N267LA',
 'N472LA',
 'N703WP',
 'N704WP',
 'N110LA',
 'N14LA ',
 'N21844',
 'N228LA',
 'N229LA',
 'N232LA',
 'N233LA',
 'N661PD',
 'N662PD',
 'N665PD',
 'N913WB',
 'N961SD',
 'N963SD',
 'N966SD',
 'N301FD',
 'N304FD',
 'N303FD',
 'N305FD',
 'N950LA',
 'N951LA',
 'N953LA',
 'N954LA',
 'N955LA',
 'N956LA',
 'N957LA',
 'N958LA',
 'N959LA',
 'N960LA',
 'N961LA',
 'N962LA',
 'N302FD',
 'N668PD',
 'N17LA ',
 'N18LA ',
 'N15LA ',
 'N160LA',
 'N190LA',
 'N821LA',
 'N822LA',
 'N950SG',
 'N951LB',
 'N952JH']

---

### Bobcat fire aircraft

In [35]:
bobcat_src = pd.read_csv('input/aircraft.csv')

In [38]:
master.iloc[0]

n_number                                                        N100  
serial_number                           5334                          
mfr_mdl_code                                                   7100510
eng_mfr_mdl                                                      17003
year_mfr                                                          1940
type_registrant                                                      1
name                                                       BENE MARY D
street                               PO BOX 329                       
street2                                                               
city                                                KETCHUM           
state                                                               OK
zip_code                                                    743490329 
region                                                               2
county                                                             097
countr

In [ ]:
src = master.merge(reference, left_on='mfr_mdl_code', right_on='code')